In [1]:
import pandas as pd
import json
import re
import numpy as np
import topojson as tp
import geopandas as gpd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/Users/briannagreen/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# load the TopoJSON data
with open("OR_SDs_simplified.topojson", "r") as file: topo_data = json.load(file)

In [3]:
# load the state data
state_data = pd.read_csv("SpEdMediaDistrict.csv")

In [4]:
# Check unique names from TopoJSON data
names_topojson = [feature['properties']['NAME'] for feature in topo_data['objects']['OR_SDs_merged']['geometries']]
print(names_topojson[:10])  # First 10 names for a glimpse

# Check unique institution names from state data
print(state_data['Institution Name'].unique()[:10])


['Harney County School District 4', 'Diamond School District 7', 'Double O School District 28', 'Drewsey School District 13', 'South Harney School District 33', 'Frenchglen School District 16', 'Juntura School District 12', 'Pine Creek School District 5', 'Suntex School District 10', 'Harney County Union High School District 1J']
['Adel SD 21' 'Adrian SD 61' 'Alsea SD 7J' 'Amity SD 4J' 'Annex SD 29'
 'Arlington SD 3' 'Arock SD 81' 'Ashland SD 5' 'Ashwood SD 8'
 'Astoria SD 1']


In [5]:
# Loop through each feature in the geometries of topo_data
for feature in topo_data['objects']['OR_SDs_merged']['geometries']:
    # Check if 'NAME' exists in properties
    if 'NAME' in feature['properties']:
        # Replace "School District" with an empty string
        name = feature['properties']['NAME'].replace("School District", "").strip()
        # Replace any sequence of spaces with a single space
        name = re.sub(' +', ' ', name)
        feature['properties']['NAME'] = name

        # Check unique names from TopoJSON data
names_topojson = [feature['properties']['NAME'] for feature in topo_data['objects']['OR_SDs_merged']['geometries']]
print(names_topojson[:10])

['Harney County 4', 'Diamond 7', 'Double O 28', 'Drewsey 13', 'South Harney 33', 'Frenchglen 16', 'Juntura 12', 'Pine Creek 5', 'Suntex 10', 'Harney County Union High 1J']


In [6]:
# Remove "SD" from state data
state_data['Institution Name'] = state_data['Institution Name'].str.replace(" SD", "", case=False)

# Check unique institution names from state data
print(state_data['Institution Name'].unique()[:10])

['Adel 21' 'Adrian 61' 'Alsea 7J' 'Amity 4J' 'Annex 29' 'Arlington 3'
 'Arock 81' 'Ashland 5' 'Ashwood 8' 'Astoria 1']


In [7]:
# Convert the relevant portion of topo_data to a DataFrame
topo_df = pd.DataFrame([{
    'NAME': feature['properties']['NAME'],
    'geometry_index': index  # Store the index to later update topo_data
} for index, feature in enumerate(topo_data['objects']['OR_SDs_merged']['geometries'])])


In [8]:
# Merge with state_data
merged_df = topo_df.merge(state_data, left_on='NAME', right_on='Institution Name', how='outer')

In [9]:
# Find non-matching entries in both datasets and print first few columns

non_matching_topojson = merged_df[merged_df['Institution Name'].isna()]

print("Non-matching from TopoJSON:", non_matching_topojson.iloc[:, :3])

Non-matching from TopoJSON:                               NAME  geometry_index  Institution ID
13               Yamhill-Carlton 1            13.0             NaN
19                          Ione 2            19.0             NaN
20                  North Wasco 21            20.0             NaN
24               Greater Albany 8J            24.0             NaN
33               Athena-Weston 29J            33.0             NaN
39   Bend-La Pine Administrative 1            39.0             NaN
41                     Blachly 090            41.0             NaN
43             Brookings-Harbor 17            43.0             NaN
64     Crow-Applegate-Lorane Sd 66            64.0             NaN
68                   Days Creek 15            68.0             NaN
90               Gresham-Barlow 1J            90.0             NaN
94             Hood River County 1            94.0             NaN
103                   Three Rivers           103.0             NaN
109                     Lakeview 7

In [10]:
non_matching_state = merged_df[merged_df['NAME'].isna()]

print("Non-matching from state data:", non_matching_state.iloc[:, :4])

Non-matching from state data:     NAME  geometry_index  Institution ID               Institution Name
198  NaN             NaN          2208.0             Athena-Weston 29RJ
199  NaN             NaN          1976.0   Bend-LaPine Administrative 1
200  NaN             NaN          2095.0                     Blachly 90
201  NaN             NaN          1974.0           Brookings-Harbor 17C
202  NaN             NaN          2089.0       Crow-Applegate-Lorane 66
203  NaN             NaN          1993.0              Douglas County 15
204  NaN             NaN          1991.0               Douglas County 4
205  NaN             NaN          2100.0       Greater Albany Public 8J
206  NaN             NaN          2183.0             Gresham-Barlow 10J
207  NaN             NaN          2024.0              Hood River County
208  NaN             NaN          3997.0                        Ione R2
209  NaN             NaN          2059.0                  Lake County 7
210  NaN             NaN          

In [11]:
# Manually match unmatched entries

# Yamhill Carlton 1
# Find the rows to match
row_topo = merged_df[merged_df['NAME'] == 'Yamhill-Carlton 1']
row_state = merged_df[merged_df['Institution Name'] == 'Yamhill Carlton 1']

# Combine the data
for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

# Drop the unmatched row
merged_df.drop(row_state.index, inplace=True)


# Ione R2
row_topo = merged_df[merged_df['NAME'] == 'Ione 2']
row_state = merged_df[merged_df['Institution Name'] == 'Ione R2']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# North Wasco County 21
row_topo = merged_df[merged_df['NAME'] == 'North Wasco 21']
row_state = merged_df[merged_df['Institution Name'] == 'North Wasco County 21']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Greater Albany Public 8J
row_topo = merged_df[merged_df['NAME'] == 'Greater Albany 8J']
row_state = merged_df[merged_df['Institution Name'] == 'Greater Albany Public 8J']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Athena-Weston 29RJ
row_topo = merged_df[merged_df['NAME'] == 'Athena-Weston 29J']
row_state = merged_df[merged_df['Institution Name'] == 'Athena-Weston 29RJ']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Bend-LaPine Administrative 1
row_topo = merged_df[merged_df['NAME'] == 'Bend-La Pine Administrative 1']
row_state = merged_df[merged_df['Institution Name'] == 'Bend-LaPine Administrative 1']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Blachly 90
row_topo = merged_df[merged_df['NAME'] == 'Blachly 090']
row_state = merged_df[merged_df['Institution Name'] == 'Blachly 90']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Brookings-Harbor 17C
row_topo = merged_df[merged_df['NAME'] == 'Brookings-Harbor 17']
row_state = merged_df[merged_df['Institution Name'] == 'Brookings-Harbor 17C']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Crow-Applegate-Lorane 66
row_topo = merged_df[merged_df['NAME'] == 'Crow-Applegate-Lorane Sd 66']
row_state = merged_df[merged_df['Institution Name'] == 'Crow-Applegate-Lorane 66']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Gresham-Barlow 10J
row_topo = merged_df[merged_df['NAME'] == 'Gresham-Barlow 1J']
row_state = merged_df[merged_df['Institution Name'] == 'Gresham-Barlow 10J']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Hood River County
row_topo = merged_df[merged_df['NAME'] == 'Hood River County 1']
row_state = merged_df[merged_df['Institution Name'] == 'Hood River County']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Three Rivers/Josephine County
row_topo = merged_df[merged_df['NAME'] == 'Three Rivers']
row_state = merged_df[merged_df['Institution Name'] == 'Three Rivers/Josephine County']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Lake County 7
row_topo = merged_df[merged_df['NAME'] == 'Lakeview 7']
row_state = merged_df[merged_df['Institution Name'] == 'Lake County 7']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Medford 549C
row_topo = merged_df[merged_df['NAME'] == 'Medford 549']
row_state = merged_df[merged_df['Institution Name'] == 'Medford 549C']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Milton-Freewater Unified 7
row_topo = merged_df[merged_df['NAME'] == 'Milton-Freewater 7']
row_state = merged_df[merged_df['Institution Name'] == 'Milton-Freewater Unified 7']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Mt Angel 91
row_topo = merged_df[merged_df['NAME'] == 'Mount Angel 91']
row_state = merged_df[merged_df['Institution Name'] == 'Mt Angel 91']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Ontario 8C
row_topo = merged_df[merged_df['NAME'] == 'Ontario 8']
row_state = merged_df[merged_df['Institution Name'] == 'Ontario 8C']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Pine Eagle 61
row_topo = merged_df[merged_df['NAME'] == 'Pine-Eagle 61']
row_state = merged_df[merged_df['Institution Name'] == 'Pine Eagle 61']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Port Orford-Langlois 2CJ
row_topo = merged_df[merged_df['NAME'] == 'Port Orford-Langlois 2J']
row_state = merged_df[merged_df['Institution Name'] == 'Port Orford-Langlois 2CJ']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Sherman County
row_topo = merged_df[merged_df['NAME'] == 'Sherman 1']
row_state = merged_df[merged_df['Institution Name'] == 'Sherman County']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# South Lane 45J3
row_topo = merged_df[merged_df['NAME'] == 'South Lane 45J']
row_state = merged_df[merged_df['Institution Name'] == 'South Lane 45J3']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# St Helens 502
row_topo = merged_df[merged_df['NAME'] == 'St. Helens 502']
row_state = merged_df[merged_df['Institution Name'] == 'St Helens 502']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# St Paul 45
row_topo = merged_df[merged_df['NAME'] == 'St. Paul 45']
row_state = merged_df[merged_df['Institution Name'] == 'St Paul 45']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Ukiah 80R
row_topo = merged_df[merged_df['NAME'] == 'Ukiah 80']
row_state = merged_df[merged_df['Institution Name'] == 'Ukiah 80R']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Umatilla 6R
row_topo = merged_df[merged_df['NAME'] == 'Umatilla 6']
row_state = merged_df[merged_df['Institution Name'] == 'Umatilla 6R']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# West Linn-Wilsonville 3J
row_topo = merged_df[merged_df['NAME'] == 'West Linn 3J']
row_state = merged_df[merged_df['Institution Name'] == 'West Linn-Wilsonville 3J']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Douglas County 15
row_topo = merged_df[merged_df['NAME'] == 'Days Creek 15']
row_state = merged_df[merged_df['Institution Name'] == 'Douglas County 15']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Douglas County 4
row_topo = merged_df[merged_df['NAME'] == 'Roseburg 4']
row_state = merged_df[merged_df['Institution Name'] == 'Douglas County 4']

for column in merged_df.columns:
    if pd.isna(row_topo[column].iloc[0]) and not pd.isna(row_state[column].iloc[0]):
        merged_df.loc[row_topo.index, column] = row_state[column].iloc[0]

merged_df.drop(row_state.index, inplace=True)


# Reset index
merged_df.reset_index(drop=True, inplace=True)

In [12]:
# Find remaining non-matching entries

non_matching_topojson = merged_df[merged_df['Institution Name'].isna()]

print("Non-matching from TopoJSON:", non_matching_topojson.iloc[:, :3])

Non-matching from TopoJSON:              NAME  geometry_index  Institution ID
116  McDermitt 51           116.0             NaN
197   Not Defined           197.0             NaN


In [13]:
non_matching_state = merged_df[merged_df['NAME'].isna()]

print("Non-matching from state data:", non_matching_state.iloc[:, :4])

Non-matching from state data:     NAME  geometry_index  Institution ID   Institution Name
198  NaN             NaN          3477.0  ODE YCEP District


In [14]:
print(merged_df.head(6))

              NAME  geometry_index  Institution ID Institution Name  \
0  Harney County 4             0.0          2015.0  Harney County 4   
1        Diamond 7             1.0          2017.0        Diamond 7   
2      Double O 28             2.0          2021.0      Double O 28   
3       Drewsey 13             3.0          2019.0       Drewsey 13   
4  South Harney 33             4.0          2022.0  South Harney 33   
5    Frenchglen 16             5.0          2020.0    Frenchglen 16   

  Total Student Count LRE Students >80% LRE Students >80% Target  \
0                  98            98.98%           76.00% or more   
1                   *                 *           76.00% or more   
2                   *                 *           76.00% or more   
3                   *                 *           76.00% or more   
4                   *                 *           76.00% or more   
5                   *                 *           76.00% or more   

  LRE Students <40% LRE S

In [15]:
# Convert relevant columns to numeric type
cols_to_convert = ['Total Student Count', 'LRE Students >80%', 'LRE Students <40%', 'LRE Students Separate Settings', 'IEP 4Yr Cohort Grad 18-19', 'IEP Dropout 18-19', 'Higher Ed/Training/Employed']

for col in cols_to_convert:
    # Replace '%' and '>' signs with an empty string
    merged_df[col] = pd.to_numeric(merged_df[col].str.replace('[%><]', '', regex=True), errors='coerce')

# # Replace any NaN values with NaN
# merged_df[cols_to_convert] = merged_df[cols_to_convert].fillna(np.nan)

In [16]:
print(merged_df.head(6))

              NAME  geometry_index  Institution ID Institution Name  \
0  Harney County 4             0.0          2015.0  Harney County 4   
1        Diamond 7             1.0          2017.0        Diamond 7   
2      Double O 28             2.0          2021.0      Double O 28   
3       Drewsey 13             3.0          2019.0       Drewsey 13   
4  South Harney 33             4.0          2022.0  South Harney 33   
5    Frenchglen 16             5.0          2020.0    Frenchglen 16   

   Total Student Count  LRE Students >80% LRE Students >80% Target  \
0                 98.0              98.98           76.00% or more   
1                  NaN                NaN           76.00% or more   
2                  NaN                NaN           76.00% or more   
3                  NaN                NaN           76.00% or more   
4                  NaN                NaN           76.00% or more   
5                  NaN                NaN           76.00% or more   

   LRE Stud

In [17]:
# Add column for percent of students who are in regular class >40% and <80%
merged_df['LRE Students >40% <80%'] = 100 - merged_df['LRE Students >80%'] - merged_df['LRE Students <40%'] - merged_df['LRE Students Separate Settings']

In [18]:
# Trim trailing spaces off of alerts columns
# Columns to trim
cols_to_trim = [
    'SuspExplFg',
    'SuspExplRaceEthnicityFg',
    'DisPrptnRprsntnFg',
    'DisPrptnRprsntnDsbltyFg'
]

# Trim trailing spaces for the specified columns
merged_df[cols_to_trim] = merged_df[cols_to_trim].apply(lambda x: x.str.rstrip())

In [19]:
# Drop the "ODE YCEP District" since there are no students or geographies recorded
merged_df = merged_df.dropna(subset=['geometry_index'])

# Drop other NaN rows
merged_df = merged_df.dropna(subset=['Institution ID'])

# Convert the geometry_index column to integers instead of float
merged_df.loc[:, 'geometry_index'] = merged_df['geometry_index'].astype(int)

# Convert '*' values to "null"
merged_df.replace('*', np.nan, inplace=True)

In [20]:
print(merged_df.head(6))

              NAME  geometry_index  Institution ID Institution Name  \
0  Harney County 4               0          2015.0  Harney County 4   
1        Diamond 7               1          2017.0        Diamond 7   
2      Double O 28               2          2021.0      Double O 28   
3       Drewsey 13               3          2019.0       Drewsey 13   
4  South Harney 33               4          2022.0  South Harney 33   
5    Frenchglen 16               5          2020.0    Frenchglen 16   

   Total Student Count  LRE Students >80% LRE Students >80% Target  \
0                 98.0              98.98           76.00% or more   
1                  NaN                NaN           76.00% or more   
2                  NaN                NaN           76.00% or more   
3                  NaN                NaN           76.00% or more   
4                  NaN                NaN           76.00% or more   
5                  NaN                NaN           76.00% or more   

   LRE Stud

In [21]:
# Update the topo_data with the merged data
for _, row in merged_df.iterrows():
    # Fetch the geometry using the stored index
    geometry = topo_data['objects']['OR_SDs_merged']['geometries'][row['geometry_index']]
    # Update properties with the merged data
    # Assuming state_data has columns 'Col1', 'Col2', etc. which you want to add to topo_data
    geometry['properties'].update({
        'Institution Name': row['Institution Name'],
        'Total Student Count': row['Total Student Count'],
        'LRE Students >80%': row['LRE Students >80%'],
        'LRE Students >40% <80%': row['LRE Students >40% <80%'],
        'LRE Students <40%': row['LRE Students <40%'],
        'LRE Students Separate Settings': row['LRE Students Separate Settings'],
        'SuspExplFg': row['SuspExplFg'],
        'SuspExplRaceEthnicityFg': row['SuspExplRaceEthnicityFg'],
        'DisPrptnRprsntnFg': row['DisPrptnRprsntnFg'],
        'DisPrptnRprsntnDsbltyFg': row['DisPrptnRprsntnDsbltyFg'],
        'IEP 4Yr Cohort Grad 18-19': row['IEP 4Yr Cohort Grad 18-19'], 
        'IEP Dropout 18-19': row['IEP Dropout 18-19'], 
        'Higher Ed/Training/Employed': row['Higher Ed/Training/Employed'],
    })

In [22]:
# Sort district geometries & data by alphabetical order
sorted_geometries = sorted(topo_data['objects']['OR_SDs_merged']['geometries'], 
                           key=lambda x: x['properties'].get('Institution Name', ''))

# Assign the sorted geometries back to the topo_data
topo_data['objects']['OR_SDs_merged']['geometries'] = sorted_geometries

In [23]:
# Convert all instances of NaN to None
def convert_nan(obj):
    if isinstance(obj, list):
        return [convert_nan(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: convert_nan(value) for key, value in obj.items()}
    else:
        return None if obj is np.nan or str(obj).lower() == 'nan' else obj
    
converted_data = convert_nan(topo_data)

In [24]:
print(converted_data['objects']['OR_SDs_merged']['geometries'][10])

{'type': 'MultiPolygon', 'arcs': [[[144, 145, 146, 147]]], 'properties': {'STATEFP': '41', 'ELSDLEA': None, 'GEOID': '4101590', 'NAME': 'Ashwood 8', 'LSAD': '00', 'LOGRADE': 'KG', 'HIGRADE': '12', 'MTFCC': 'G5420', 'SDTYP': None, 'FUNCSTAT': 'E', 'ALAND': 1110402084, 'AWATER': 319758, 'INTPTLAT': '+44.6613371', 'INTPTLON': '-120.6140461', 'SCSDLEA': None, 'UNSDLEA': '01590', 'layer': 'tl_2022_41_unsd', 'path': '/Users/briannagreen/Desktop/Oregon 22 data/tl_2022_41_unsd/tl_2022_41_unsd.shp', 'Institution Name': 'Ashwood 8', 'Total Student Count': None, 'LRE Students >80%': None, 'LRE Students >40% <80%': None, 'LRE Students <40%': None, 'LRE Students Separate Settings': None, 'SuspExplFg': 'No', 'SuspExplRaceEthnicityFg': 'No', 'DisPrptnRprsntnFg': 'No', 'DisPrptnRprsntnDsbltyFg': 'No', 'IEP 4Yr Cohort Grad 18-19': None, 'IEP Dropout 18-19': None, 'Higher Ed/Training/Employed': None}}


In [25]:
print(converted_data["objects"].keys())

dict_keys(['OR_SDs_merged'])


In [26]:
# Path to output file
output_file_path = "oregon_data.topojson"

# Export
with open(output_file_path, 'w') as f:
    json.dump(converted_data, f)